1️⃣ Chargement des données et Prétraitement

In [1]:
import pandas as pd
import os
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np
import zipfile

# 🔗 Monter Google Drive
drive.mount('/content/drive')

# 📂 Définir le chemin du fichier ZIP
zip_path = "/content/drive/My Drive/Voitures.zip"

# 📢 Vérification de l'existence du fichier ZIP
if os.path.exists(zip_path):
    print("✅ Fichier ZIP trouvé :", zip_path)
else:
    raise FileNotFoundError("❌ Le fichier ZIP n'existe pas ! Vérifiez le chemin.")

# 🗜 Extraire le fichier ZIP
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/drive/My Drive/")
        print("✅ Fichier ZIP extrait avec succès !")
except Exception as e:
    print("❌ Erreur lors de l'extraction du fichier ZIP :", e)

# 📂 Définir le chemin du fichier CSV extrait
csv_path = "/content/drive/My Drive/vehicles.csv"

# 📢 Vérification de l'existence du fichier CSV
if os.path.exists(csv_path):
    print("✅ Fichier CSV trouvé :", csv_path)
else:
    raise FileNotFoundError("❌ Le fichier CSV n'existe pas après l'extraction ! Vérifiez le chemin.")

# 📊 Charger le fichier CSV correctement
try:
    df = pd.read_csv(csv_path, encoding='utf-8', sep=',', dtype=str, keep_default_na=True, on_bad_lines='skip')
    print("✅ Fichier chargé avec succès !")

    # 🧐 Aperçu des premières lignes
    print(df.head())

except Exception as e:
    print("❌ Erreur lors du chargement :", e)


Mounted at /content/drive
✅ Fichier ZIP trouvé : /content/drive/My Drive/Voitures.zip
✅ Fichier ZIP extrait avec succès !
✅ Fichier CSV trouvé : /content/drive/My Drive/vehicles.csv
✅ Fichier chargé avec succès !
           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price year  \
0                prescott    https://prescott.craigslist.org   6000  NaN   
1            fayetteville       https://fayar.craigslist.org  11900  NaN   
2            florida keys        https://keys.craigslist.org  21000  NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500  NaN  

2️⃣ Nettoyage des Données

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 🗂️ Colonnes à utiliser
columns_to_keep = [
    "paint_color", "year", "manufacturer", "model", "condition", "cylinders", "transmission", "price"
]

# 🌟 Garder uniquement les colonnes nécessaires
df = df[columns_to_keep]

# 🚨 Remplacement des valeurs manquantes par des valeurs par défaut adaptées
default_values = {
    "paint_color": "unknown",
    "year": 2000,
    "manufacturer": "unknown",
    "model": "unknown",
    "condition": "unknown",
    "cylinders": 0,
    "transmission": "unknown",
    "price": 0
}
df.fillna(default_values, inplace=True)

# 🏷️ Conversion des colonnes numériques en types appropriés
df["year"] = pd.to_numeric(df["year"], errors='coerce').fillna(2000).astype(int)
df["cylinders"] = pd.to_numeric(df["cylinders"], errors='coerce').fillna(0).astype(int)
df["price"] = pd.to_numeric(df["price"], errors='coerce').fillna(0).astype(float)

# 🔠 Encodage des colonnes catégorielles avec LabelEncoder
categorical_columns = ["paint_color", "manufacturer", "model", "condition", "transmission"]
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# 🔢 Normalisation des colonnes numériques (year, cylinders, price)
numeric_columns = ["year", "cylinders", "price"]
scaler = MinMaxScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 🚀 Séparer les features (X) et la target (y)
X = df.drop(columns=["price"])
y = df["price"]

# 🔀 Division en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📢 Vérification des tailles
print("Taille des données d'entraînement :", X_train.shape)
print("Taille des données de test :", X_test.shape)


Taille des données d'entraînement : (341504, 7)
Taille des données de test : (85376, 7)


3️⃣ Transformation des Données

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# 🗂️ Liste des colonnes à utiliser
columns_to_keep = [
    "region", "year", "manufacturer", "model", "condition", "cylinders",
    "fuel", "odometer", "title_status", "transmission", "drive", "size",
    "type", "paint_color", "state", "price"
]

print("Colonnes disponibles :", df.columns)


# 🌟 Garder uniquement les colonnes nécessaires
# 🗂️ Vérifier et filtrer les colonnes existantes
existing_columns = df.columns.intersection(columns_to_keep)
df = df[existing_columns]

# 🚨 Remplacement des valeurs manquantes par des valeurs par défaut adaptées
default_values = {
    "region": "unknown",
    "year": df["year"].mode()[0] if df["year"].dropna().shape[0] > 0 else 2000,
    "manufacturer": "unknown",
    "model": "unknown",
    "condition": "unknown",
    "cylinders": "0",
    "fuel": "unknown",
    "odometer": "0",
    "title_status": "unknown",
    "transmission": "unknown",
    "drive": "unknown",
    "size": "unknown",
    "type": "unknown",
    "paint_color": "unknown",
    "state": "unknown",
    "price": "0"
}

df.fillna(default_values, inplace=True)

# 🏷️ Conversion des colonnes en types appropriés
df["year"] = pd.to_numeric(df["year"], errors='coerce').fillna(2000).astype(int)
# df["odometer"] = pd.to_numeric(df["odometer"], errors='coerce').fillna(0).astype(float)
df["price"] = pd.to_numeric(df["price"], errors='coerce').fillna(0).astype(float)

# 📊 Vérification des valeurs manquantes
print("🔍 Nombre de valeurs manquantes après correction :\n", df.isnull().sum())

# 🔢 Sélection des colonnes numériques pour la normalisation
numeric_columns = ["year", "price"]

# 🔢 Normalisation des colonnes numériques (remplace les valeurs d'origine)
scaler = MinMaxScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 🔠 Sélection des colonnes catégorielles
categorical_columns = df.select_dtypes(include=['object']).columns

# ⚡ Réduction du nombre de catégories pour éviter l'explosion mémoire
for col in categorical_columns:
    top_categories = df[col].value_counts().index[:10]  # Garde les 10 catégories les plus courantes
    df[col] = df[col].apply(lambda x: x if x in top_categories else "other")

# 🔠 Encodage LabelEncoder pour limiter la RAM (au lieu de One-Hot)
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Stocker les encodeurs si besoin plus tard

# ⚠️ Échantillonnage pour réduire la taille des données (évite crash Colab)
df = df.sample(frac=0.5, random_state=42)  # Réduire à 50% (modulable)

# 🎉 DataFrame prêt à être utilisé
print("✅ Données prêtes après normalisation et encodage :\n", df.head())



Colonnes disponibles : Index(['paint_color', 'year', 'manufacturer', 'model', 'condition',
       'cylinders', 'transmission', 'price'],
      dtype='object')
🔍 Nombre de valeurs manquantes après correction :
 paint_color     0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
transmission    0
price           0
dtype: int64
✅ Données prêtes après normalisation et encodage :
         paint_color  year  manufacturer  model  condition  cylinders  \
100905           11   0.0            13  15258          2        0.0   
143835            0   0.0             7  11429          2        0.0   
20235            10   0.0             7  11429          6        0.0   
300734           10   0.0             4   2684          6        0.0   
316249            0   0.0             7  26075          6        0.0   

        transmission         price  
100905             2  9.898503e-06  
143835             1  7.491446e-06  
20235              0  2.098595e-05  


4️⃣ 1️⃣ Séparer les données en train/test

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# 🚀 Séparer les features (X) et la target (y)
X = df.drop(columns=["price"])
y = df["price"]

# 🔀 Division en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📢 Vérification des tailles
print("Taille des données d'entraînement :", X_train.shape)
print("Taille des données de test :", X_test.shape)

Taille des données d'entraînement : (170752, 7)
Taille des données de test : (42688, 7)


4️⃣ 2️⃣ Création du réseau de neurones

In [5]:
# 🎯 Définition du modèle
model = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),  # Couche d'entrée
    layers.Dense(128, activation="relu"),  # Couche cachée
    layers.Dense(64, activation="relu"),  # Couche cachée
    layers.Dense(1, activation="linear")  # Couche de sortie (prédiction du prix)
])

# 🔧 Compilation du modèle
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# 📜 Affichage du résumé
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 17153 (67.00 KB)
Trainable params: 17153 (67.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
!pip install -U tensorflow==2.13.0  # Ou une version plus stable
import tensorflow as tf
print(tf.__version__)  # Vérifie la version installée

2.13.0


3️⃣ Entraînement du modèle

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 📌 Définition du modèle
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Couche d'entrée
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Couche de sortie pour prédiction du prix
])

# 📌 Compilation du modèle
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# ✅ Vérification
print(model.summary())  # Vérifie la structure du modèle



# 🚀 Entraînement du modèle
history = model.fit(
    X_train, y_train,
    epochs=50,  # Nombre d'époques (peut être ajusté)
    batch_size=32,  # Taille des batchs
    validation_data=(X_test, y_test),
    verbose=1
)
# Sauvegarde du modèle
from google.colab import drive
drive.mount('/content/drive')

model.save("/content/drive/My Drive/voiture_pricing_model.h5")
model.save("/content/drive/My Drive/voiture_pricing_model.keras")
print("✅ Modèle sauvegardé avec succès !")


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               1024      
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 11393 (44.50 KB)
Trainable params: 11393 (44.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/50
5336/5336 [==============================] - 11s 2ms/step - loss: 145.2453 - mae: 1.5303 - val_loss: 0.0026 - val_mae: 0.0399
Epoch 2/

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


✅ Modèle sauvegardé avec succès !


5️⃣ Evaluation du Modele

In [ ]:
# 📊 Évaluation sur les données de test
loss, mae = model.evaluate(X_test, y_test)
print(f"📉 Perte (MSE) : {loss:.4f}")
print(f"📏 Erreur moyenne absolue (MAE) : {mae:.4f}")


334/334 [==============================] - 0s 1ms/step - loss: 6.5446e-11 - mae: 4.3530e-06
📉 Perte (MSE) : 0.0000
📏 Erreur moyenne absolue (MAE) : 0.0000


5️⃣ 1️⃣ Installation et chargement du modèle Stable Diffusion

5️⃣2️⃣ Génération d'une image à partir des caractéristiques